In [ ]:
import configparser
import pandas as pd 
from  pymongo import MongoClient


In [ ]:
config=configparser.ConfigParser()
config.read(r"C:\Users\Harshavardhan\Documents\python_tutorials\MangoDB\config.config")
url=config['mongo']['url']
client=MongoClient(url)
db= client['First']
collection=db["Unstructured"]
doc=list(collection.find())
df=pd.DataFrame(doc)

In [ ]:
df

In [ ]:
df["_id"]=df["_id"].astype(str)
project_df=df[['_id','project_id','technologies','status']]

In [ ]:
project_df

In [ ]:
client_df=pd.json_normalize(df['client'],sep='_')
client_df['project_id']=df['project_id']
client_df=client_df.rename(columns={
    "name": "client_name",
    "location_city": "city",
    "location_country": "country"
})[["project_id", "client_name", "industry", "city", "country"]]

In [ ]:
client_df

In [ ]:
client_df['client_id']=['CLI'+str(i+1).zfill(3) for i in range(len(client_df))]

In [ ]:
client_df

In [ ]:
client_df=client_df[['project_id','client_id','client_name','industry','city','country']]

In [ ]:
client_df

In [66]:
teams=df['team'].apply(pd.Series).explode('members')
teams


,project_manager,members
0,Amit Sinha,"{'name': 'Riya Mehta', 'role': 'DevOps Engineer'}"
0,Amit Sinha,"{'name': 'Sagar Patel', 'role': 'Cloud Archite..."
1,Pooja Reddy,"{'name': 'Anil Kumar', 'role': 'Data Engineer'}"
1,Pooja Reddy,"{'name': 'Nidhi Das', 'role': 'Business Analyst'}"


In [69]:
df=pd.json_normalize(df)

In [70]:
teams = df[['project_id', 'team.members']].explode('team.members')
teams = pd.concat([
    teams.drop(columns='team.members'), teams['team.members'].apply(pd.Series)],
    axis=1
)
df_pm = pd.DataFrame({'project_id': df['project_id'],'name': df['team.project_manager'],'role': 'project_manager'})
teams = pd.concat([teams, df_pm], ignore_index=True)
 

KeyError: "None of [Index(['project_id', 'team.members'], dtype='object')] are in the [columns]"

In [ ]:
rows = []
for _, row in df.iterrows():
    project_id = row["project_id"]
    team = row.get("team", {})
      
      # Add project manager
    pm = team.get("project_manager")
    if pm:
        rows.append({
            "project_id": project_id,
            "member_name": pm,
            "role": "Project Manager"
          })
      # Add team members
    members = team.get("members", [])
    for member in members:
        rows.append({
            "project_id": project_id,
            "member_name": member.get("name"),
            "role": member.get("role")
        })
team_df = pd.DataFrame(rows)

In [ ]:
team_df

In [ ]:
team_df['member_id']=['MEM'+str(i+1).zfill(3) for i in range(len(team_df))]

In [58]:
team_df

,project_id,member_id,member_name,role
0,HCL001,MEM001,Amit Sinha,Project Manager
1,HCL001,MEM002,Riya Mehta,DevOps Engineer
2,HCL001,MEM003,Sagar Patel,Cloud Architect
3,HCL002,MEM004,Pooja Reddy,Project Manager
4,HCL002,MEM005,Anil Kumar,Data Engineer
5,HCL002,MEM006,Nidhi Das,Business Analyst


In [ ]:
team_df=team_df[['project_id','member_id','member_name','role']]

In [ ]:
team_df

In [ ]:
from datetime import datetime

In [ ]:
milestone_rows=[]

for i,row in df.iterrows():
    project_id=row['project_id']
    milestones=row.get('milestones',[])

    for milestone in milestones:
        milestone_rows.append({
            'project_id':project_id,
            'milestone_name':milestone.get('name'),
            'due_date':pd.to_datetime(milestone.get("due_date"),errors='coerce')
        })

milestone_df=pd.DataFrame(milestone_rows)

In [ ]:
milestone_df

In [ ]:
milestone_df['milestone_id']=['MIL'+str(i+1).zfill(3) for i in range(len(milestone_df))]

In [ ]:
milestone_df=milestone_df[['project_id','milestone_id','milestone_name','due_date']]
milestone_df

In [71]:
# Step 1: Get basic fact table with project and client
fact_df = project_df.merge(client_df[['project_id', 'client_id']], on='project_id', how='left')

# Step 2: Add total team members per project
team_counts = team_df.groupby('project_id').size().reset_index(name='total_members')
fact_df = fact_df.merge(team_counts, on='project_id', how='left')

# Step 3: Add total milestones per project
milestone_counts = milestone_df.groupby('project_id').size().reset_index(name='total_milestones')
fact_df = fact_df.merge(milestone_counts, on='project_id', how='left')

# Step 4: Add number of technologies per project
fact_df['technologies_count'] = project_df['technologies'].apply(lambda x: len(x) if isinstance(x, list) else 0)

# Step 5: Add status from project_df (already included if you merged project_df at the start)
fact_df = fact_df[['project_id', 'client_id', 'status', 'total_members', 'total_milestones', 'technologies_count']]
fact_df = fact_df.rename(columns={"status": "project_status"})


In [72]:
fact_df

,project_id,client_id,project_status,total_members,total_milestones,technologies_count
0,HCL001,CLI001,In Progress,3,2,3
1,HCL002,CLI002,Planned,3,2,3
